In [1]:
import json
import ROOT
from analysis_framework import Analysis, ILDMC2020Dataset

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x623ee70


In [2]:
ROOT.EnableImplicitMT(12)

In [3]:
# dataset = Dataset("data/locations/miniDSTs/processed-no-higgs.txt")
# dataset = Dataset("data/locations/miniDSTs/processed-no-exc-higgs.txt")
# dataset = Dataset("data/locations/miniDSTs/processed-no-exc-higgs-reduced.txt")
# dataset = Dataset("data/locations/miniDSTs/processed-no-exc-higgs-reduced500.txt")
dataset = ILDMC2020Dataset("data/locations/miniDSTs/processed-test.txt", "data/ILD-genmeta/genmetaByID.json")

KeyError: '6f_llxyyx_eLpR'

In [ ]:
analysis = Analysis(dataset)

analysis.init_parameters([
    ("WWCategorisation.RecoCatBasic", "int", "8"),
    ("WWCategorisation.RecoCatAdvanced", "int", "8"),
    ("WWCategorisation.missE", "float", "-42"),
    ("WWCategorisation.misspT", "float", "-42"),
    ("WWCategorisation.mInv", "float", "-42"),
])

In [ ]:
# signal cut goes here:
# In this case, we want to require the charged lepton in the final state to be inside of the detector acceptance
# just take the first gen status 1 particle with a fitting pdg value and cut on theta
lep_pdg = 11
# CLD 150 mrad
# ILD ~84 mrad (IDR: 4.8 deg)
acceptance_theta = 0.084
signal_cut = f"""
std::invoke([](auto& genStat, auto& pdg, auto& px, auto& py, auto& pz, auto& m) -> bool {{
auto lepton_mask = genStat == 1 && abs(pdg) == {lep_pdg};
// abuse ArgMax to get the first set position
auto lepton_idx = ArgMax(lepton_mask);
auto lepton_lvec = ROOT::Math::PxPyPzMVector(px[lepton_idx], py[lepton_idx],
                                             pz[lepton_idx], m[lepton_idx]);
double lepton_theta = lepton_lvec.Theta();
return abs(cos(lepton_theta)) < cos({acceptance_theta});
}}, MCParticlesSkimmed.generatorStatus, MCParticlesSkimmed.PDG, MCParticlesSkimmed.momentum.x, MCParticlesSkimmed.momentum.y, MCParticlesSkimmed.momentum.z, MCParticlesSkimmed.mass)
"""

In [ ]:
analysis.set_categories({
    # TODO: actually implement the cut, but I might notice when refactoring the cut-flow...
    # "4f_sw_sl_signal": {"pattern": "4f_sw_sl", "cut": "1 == 1"},
    "4f_sw_sl_signal": {"pattern": "4f_sw_sl", "cut": signal_cut},
    "4f_sl_bkg": {"pattern": r"4f\w+sl", "cut": None }, # inverse signal cut will be applied automatically
    "4f_not_sl": {"pattern": r"4f\w+_(?:h|l)", "cut": None },
    # separate out aa_4f? super low lumi anyway
    "aa2f": {"pattern": "aa_2f", "cut": None},
    # 2f but not aa_2f
    "2f": {"pattern": "(?<!aa_)2f", "cut": None},
    "3f": {"pattern": "ea_3f|ae_3f", "cut": None},
    "5f": {"pattern": "ea_5f|ae_5f", "cut": None},
    "6f": {"pattern": "6f", "cut": None},
    # need to filter out anything ending in _h and 2f_z_eehiq
    "higgs": {"pattern": "[^_e]h", "cut": None},
})
# check if we missed any processes
print(analysis.is_complete_categorisation())

True


In [ ]:
print(json.dumps(analysis._categories, indent=2))

{
  "4f_sw_sl_signal": [
    "4f_sw_sl_eLpL_signal",
    "4f_sw_sl_eRpR_signal",
    "4f_sw_sl_eLpR_signal",
    "4f_sw_sl_eRpL_signal"
  ],
  "4f_sl_bkg": [
    "4f_sze_sl_eLpR",
    "4f_sze_sl_eRpL",
    "4f_sze_sl_eLpL",
    "4f_zz_sl_eLpR",
    "4f_zz_sl_eRpL",
    "4f_zznu_sl_eLpR",
    "4f_zznu_sl_eRpL",
    "aa_4f_zz_sl_eBpB",
    "4f_ww_sl_eRpL",
    "aa_4f_ww_sl_eBpB",
    "4f_sznu_sl_eRpL",
    "4f_sw_sl_eLpL_bkg",
    "4f_sw_sl_eRpR_bkg",
    "4f_sw_sl_eLpR_bkg",
    "4f_sw_sl_eRpL_bkg"
  ],
  "4f_not_sl": [
    "4f_zzorww_h_eLpR",
    "4f_zzorww_h_eRpL",
    "4f_zz_h_eLpR",
    "4f_zz_h_eRpL",
    "4f_ww_h_eLpR",
    "4f_ww_h_eRpL",
    "4f_sznu_l_eLpR",
    "4f_sznu_l_eRpL",
    "4f_szeorsw_l_eRpL",
    "4f_szeorsw_l_eLpR",
    "4f_szeorsw_l_eRpR",
    "4f_zznu_l_eLpR",
    "4f_zznu_l_eRpL",
    "4f_zz_l_eRpL",
    "4f_zz_l_eLpR",
    "4f_ww_l_eLpR",
    "4f_sze_l_eLpR",
    "4f_sze_l_eRpL",
    "4f_sze_l_eLpL",
    "4f_sze_l_eRpR",
    "aa_4f_zzorww_l_eBpB",
    "4f_sw_l_

In [ ]:
# needed for the .size() calls... alternative would probably be to .Alias the @size columns
ROOT.gInterpreter.Declare("#include <podio/ObjectID.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/ReconstructedParticleData.h>")
analysis.Define("n_isomuons", "IsolatedMuons_objIdx.size()")
analysis.Define("n_isoelectrons", "IsolatedElectrons_objIdx.size()")
analysis.Define("RecoCatBasic", "params_WWCategorisation_RecoCatBasic")
analysis.Define("RecoCatAdvanced", "params_WWCategorisation_RecoCatAdvanced")
analysis.Define("n_charged_PFOs", "Sum(abs(PandoraPFOs.charge) == 1.)")
analysis.Define("n_R2jets", "Refined2Jets.size()")
make_lvec = lambda coll_name: f"ROOT::VecOps::Construct<ROOT::Math::PxPyPzEVector>({coll_name}.momentum.x, {coll_name}.momentum.y, {coll_name}.momentum.z, {coll_name}.energy)"
analysis.Define("R2jet_lvecs", make_lvec("Refined2Jets"))
analysis.Define("M_jj", "ROOT::VecOps::Sum(R2jet_lvecs, ROOT::Math::PxPyPzEVector()).mass()")
analysis.Define("PVertex_ndf", "PrimaryVertex.ndf")
analysis.Define("PVertex_chi2", "PrimaryVertex.chi2")
analysis.Define("PVertex_chi2ndf", "PVertex_chi2 / PVertex_ndf")

In [ ]:
analysis.book_histogram_1D("RecoCatBasic", "RecoCatBasic", ("", ";RecoCatBasic", 8, 0., 8.))
analysis.book_histogram_1D("RecoCatAdvanced", "RecoCatAdvanced", ("", ";RecoCatAdvanced", 8, 0., 8.))

analysis.add_filter("RecoCatAdvanced == 2", "RecoCatAdvanced == 2")

In [ ]:

analysis.book_histogram_1D("n_isoelectrons", "n_isoelectrons", ("", ";n_isoelectrons", 5, 0., 5.))
analysis.book_histogram_1D("n_charged_PFOs", "n_charged_PFOs", ("", ";n_charged_PFOs", 50, 0., 50.))

analysis.add_filter("n_charged_PFOs >= 10", "n_charged_PFOs geq 10")
analysis.add_filter("n_isoelectrons == 1", "1 iso electron")

analysis.book_histogram_1D("misspT", "params_WWCategorisation_misspT", ("", ";misspT", 100, 0., 100.))

analysis.add_filter("params_WWCategorisation_misspT >= 10", "misspT geq 10")

analysis.book_histogram_1D("PVertex_ndf", "PVertex_ndf", ("", ";PVertex_ndf", 50, 1., 101.))

analysis.book_histogram_1D("missE", "params_WWCategorisation_missE", ("", ";missE", 300, -50., 250.))
analysis.book_histogram_1D("mInv", "params_WWCategorisation_mInv", ("", ";mInv", 100, 40., 140.))
analysis.book_histogram_1D("M_jj", "M_jj", ("", ";M_jj", 100, 32., 132.))
analysis.book_histogram_1D("n_isomuons", "n_isomuons", ("", ";n_isomuons", 5, 0., 5.))
analysis.book_histogram_1D("PVertex_chi2", "PVertex_chi2", ("", ";PVertex_chi2", 75, 0., 150.))
analysis.book_histogram_1D("PVertex_chi2ndf", "PVertex_chi2ndf", ("", ";PVertex_chi2/ndf", 20, 0., 10.))

In [ ]:
analysis.book_reports()

In [ ]:
# analysis.book_snapshots("events", "data/snapshots", ["PandoraPFOs"])
# analysis.book_snapshots("events", "root://eosuser.cern.ch//eos/user/l/lreichen/snapshot-test/reduced", ["PandoraPFOs"])
# analysis.book_snapshots("events", "root://eosuser.cern.ch//eos/user/l/lreichen/snapshot-test/reduced500", ["PandoraPFOs"])
# Have also no dict for these RVecs thanks to root having forgotten it...
# analysis.book_snapshots("events", "root://eosuser.cern.ch//eos/user/l/lreichen/snapshot-test-full/full-zstd-9", column_list=r"^(?!R2jet_lvecs)\w+$", no_rvec=True)

In [ ]:
%%time
analysis.run()

CPU times: user 6min 43s, sys: 1min 5s, total: 7min 49s
Wall time: 5min 26s
OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x26a1cbb0


In [ ]:
analysis.print_reports()

       4f_sw_sl_signal             4f_sl_bkg             4f_not_sl                  aa2f                    2f                    3f                    5f                    6f                 higgs
        13143286 (2e+04)         8121644 (9e+03)        73686097 (1e+05)      7613111686 (1e+07)      1272531834 (2e+06)       140022808 (4e+05)           31964 (3e+02)             948 (9e+00)         1050064 (1e+03) All
        10940484 (2e+04)         1550440 (4e+03)         1213754 (1e+04)          275799 (6e+04)        42894777 (4e+05)         1027338 (3e+04)            1479 (6e+01)              31 (2e+00)           45061 (2e+02) RecoCatAdvanced == 2
        10913065 (2e+04)         1539888 (4e+03)          748308 (1e+04)          259125 (6e+04)        27187986 (3e+05)          884465 (3e+04)            1422 (6e+01)              31 (2e+00)           44982 (2e+02) n_charged_PFOs geq 10
        10886001 (2e+04)          865380 (3e+03)          552381 (9e+03)          253838 (6e+04)       

In [ ]:
# unfortunately the jsroot plots break all the time, maybe it is just too much for them
# %jsroot on
# 0 - hadronic (both W decay hadronically)
# 1 - invisible semileptonic (one W decays hadronically, but the lepton is not within detector acceptance/detected)
# 2 - semileptonic electron
# 3 - semileptonic muon
# 4 - semileptonic tauon
# 5 - leptonic (both W decay leptonically)
# 6 - other, 7- broken?
analysis.draw_histogram("RecoCatBasic")
analysis.draw_histogram("RecoCatAdvanced")
analysis.draw_histogram("n_charged_PFOs")
analysis.draw_histogram("n_isoelectrons")

Error in callback <bound method CaptureDrawnPrimitives._post_execute of <JupyROOT.helpers.utils.CaptureDrawnPrimitives object at 0x7f11c1f22190>> (for post_execute), with arguments args (),kwargs {}:


AttributeError: <class cppyy.gbl.TWebCanvas at 0x3555ea50> has no attribute 'CreateCanvasJSON'. Full details:
  type object 'TWebCanvas' has no attribute 'CreateCanvasJSON'
  'TWebCanvas::CreateCanvasJSON' is not a known C++ class
  'CreateCanvasJSON' is not a known C++ template
  'CreateCanvasJSON' is not a known C++ enum

In [ ]:
analysis.draw_histogram("misspT")

Error in callback <bound method CaptureDrawnPrimitives._post_execute of <JupyROOT.helpers.utils.CaptureDrawnPrimitives object at 0x7f11c1f22190>> (for post_execute), with arguments args (),kwargs {}:


AttributeError: <class cppyy.gbl.TWebCanvas at 0x356057c0> has no attribute 'CreateCanvasJSON'. Full details:
  type object 'TWebCanvas' has no attribute 'CreateCanvasJSON'
  'TWebCanvas::CreateCanvasJSON' is not a known C++ class
  'CreateCanvasJSON' is not a known C++ template
  'CreateCanvasJSON' is not a known C++ enum

In [ ]:
analysis.draw_histogram("PVertex_ndf")

analysis.draw_histogram("missE")
analysis.draw_histogram("mInv")
analysis.draw_histogram("M_jj")
analysis.draw_histogram("n_isomuons")

Error in callback <bound method CaptureDrawnPrimitives._post_execute of <JupyROOT.helpers.utils.CaptureDrawnPrimitives object at 0x7f11c1f22190>> (for post_execute), with arguments args (),kwargs {}:


AttributeError: <class cppyy.gbl.TWebCanvas at 0x3551f7d0> has no attribute 'CreateCanvasJSON'. Full details:
  type object 'TWebCanvas' has no attribute 'CreateCanvasJSON'
  'TWebCanvas::CreateCanvasJSON' is not a known C++ class
  'CreateCanvasJSON' is not a known C++ template
  'CreateCanvasJSON' is not a known C++ enum

In [ ]:
analysis.draw_histogram("PVertex_chi2")
analysis.draw_histogram("PVertex_chi2ndf")

Error in callback <bound method CaptureDrawnPrimitives._post_execute of <JupyROOT.helpers.utils.CaptureDrawnPrimitives object at 0x7f11c1f22190>> (for post_execute), with arguments args (),kwargs {}:


AttributeError: <class cppyy.gbl.TWebCanvas at 0x34226b50> has no attribute 'CreateCanvasJSON'. Full details:
  type object 'TWebCanvas' has no attribute 'CreateCanvasJSON'
  'TWebCanvas::CreateCanvasJSON' is not a known C++ class
  'CreateCanvasJSON' is not a known C++ template
  'CreateCanvasJSON' is not a known C++ enum

In [ ]:
# TODO: plot the 5 (10?) differential cross sections down here

Error in callback <bound method CaptureDrawnPrimitives._post_execute of <JupyROOT.helpers.utils.CaptureDrawnPrimitives object at 0x7f11c1f22190>> (for post_execute), with arguments args (),kwargs {}:


AttributeError: <class cppyy.gbl.TWebCanvas at 0x357a1700> has no attribute 'CreateCanvasJSON'. Full details:
  type object 'TWebCanvas' has no attribute 'CreateCanvasJSON'
  'TWebCanvas::CreateCanvasJSON' is not a known C++ class
  'CreateCanvasJSON' is not a known C++ template
  'CreateCanvasJSON' is not a known C++ enum